# P2B One vs ALL logistic regression

In [47]:
############################# IMPORTS #############################
from nltk.tokenize import word_tokenize
from nltk.corpus import brown
from collections import defaultdict
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import re


In [48]:
############################# INITIALIZATIONS #############################
data = []
raw_data = brown.tagged_words()

Vocab = set()
tag_list = set()
tag_num_dict = {}


In [67]:
############################# DATA PREPROCESSING #############################
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

def pre_process(data):
    data1 = []

    for i in range(len(data[0:2000])):
        if hasNumbers(data[i][0]):
            continue
        
        elif data[i][0] == '.':
            data1.append(('.','.'))
            
        elif len(data[i][1]) == 1:
            continue
        
        elif data[i][0]==data[i][1]:
            continue
        else:
            if data[i][1].find('-')!=-1:
                temp = data[i][1].split("-")
                temp[0] = temp[0].replace("*","")
                temp[0] = temp[0].replace("$","")
                temp[0] = temp[0].replace(")","")
                temp[0] = temp[0].replace("(","")
                temp[0] = temp[0].replace(":","")
                temp[0] = temp[0].replace(",","")
                temp[0] = temp[0].replace(".","")
                
                if len(temp[0]) > 1:
                    data1.append((data[i][0],temp[0][:2]))
            
            elif data[i][1].find('+')!=-1:
                temp = data[i][1].split("+")
                temp[0] = temp[0].replace("*","")
                temp[0] = temp[0].replace("$","")
                temp[0] = temp[0].replace(")","")
                temp[0] = temp[0].replace("(","")
                temp[0] = temp[0].replace(":","")
                temp[0] = temp[0].replace(".","")
                temp[0] = temp[0].replace(",","")
                
                if len(temp[0]) > 1:
                    data1.append((data[i][0],temp[0][:2]))   
            else:
                temp = data[i][1].replace("*","")
                temp = temp.replace("*","")
                temp = temp.replace("$","")
                temp = temp.replace(")","")
                temp = temp.replace("(","")
                temp = temp.replace(":","")
                temp = temp.replace(".","")
                temp = temp.replace(",","")
                
                if len(temp) > 1:
                    data1.append((data[i][0],temp[:2]))
    data1.pop()
    data1.pop()
    processed_data = []
    temp = []
    
    for i in range(len(data1)):
        if data1[i][0] == '.':
            temp.append(tuple(data1[i]))
            processed_data.append(temp)
            temp = []
        else:
            temp.append(data1[i])
    
    return processed_data

In [68]:
##################### Populate dictionaries ##############

def build_dicts(sentences):
    global Vocab
    global tag_list , tag_num_dict
    
    for sentence in sentences:
        
        for words in sentence:
            Vocab.add(words[0])
            tag_list.add(words[1])
    index=0  
    for i in tag_list:
        tag_num_dict[i]=index
        index+=1
    tag_num_dict["UK"] = index    


In [69]:
######################### FEATURE EXTRACTION #########################
def get_feature(token, token_index, sent):
    #global token_feature
    token_feature = { 'token'             : token,
                    'is_first'          : token_index == 0,
                    'is_last'           : token_index == len(sent)-1,

                    'is_capitalized'    : token[0].upper() == token[0],
                    'is_all_capitalized': token.upper() == token,
                    'is_capitals_inside': token[1:].lower() != token[1:],
                    'is_numeric'        : token.isdigit(),

                    'prefix-1'          : token[0],
                    'prefix-2'          : '' if len(token) < 2  else token[:1],

                    'suffix-1'          : token[-1],
                    'suffix-2'          : '' if len(token) < 2  else token[-2:],

                    'prev-token'        : '' if token_index == 0     else sent[token_index - 1][0],
                    '2-prev-token'      : '' if token_index <= 1     else sent[token_index - 2][0],

                    'next-token'        : '' if token_index == len(sent) - 1     else sent[token_index + 1][0],
                    '2-next-token'      : '' if token_index >= len(sent) - 2     else sent[token_index + 2][0]
                    }

    return  token_feature

def feature_extraction(pdata):
    global tag_num_dict
    features = []
    pos_labels = []
    
    for sent in pdata:
        for token_index, token_pair in enumerate(sent):
            features.append(get_feature(token_pair[0],token_index,sent))
            pos_labels.append(tag_num_dict.get(token_pair[1],-1))
    
    return features,pos_labels
    

In [70]:
############################# ONE vs ALL LOGISTIC REGRESSION #############################
def costFunction(features, target, weights):
    cost = np.array([],dtype=np.float128)
    scores = np.array([],dtype=np.float128)
    scores = np.dot(features, weights)
    hypothesis = sigmoid(scores)
    cost = np.sum((-target*np.log(hypothesis))-((1-target)*np.log(1-hypothesis)))   
    return cost

def sigmoid(scores):
    return 1/(1 + np.exp(-scores))

def logistic_Regression(wordFeature,targetPOS,num_epochs,learning_rate):
    targetPOS = np.array(targetPOS)
    num_classes = len(tag_list)
    num_feature = wordFeature.shape[1]
    classifier = np.zeros(shape=(num_classes+1,num_feature),dtype=np.float128)
    
    print("Training for label           Cost Function\n")
    for c in range(0,num_classes+1):
        #print("Training for label: ",c)
        targetLabel =  (targetPOS==c).astype(int)
        weights = np.zeros(wordFeature.shape[1],dtype=np.float128)
        
        for epoch in range(num_epochs):
            scores = np.dot(wordFeature,weights)
            predictions = sigmoid(scores)
            
            output_error = targetLabel - predictions
            gradient = np.dot(wordFeature.T,output_error)
            weights += (learning_rate * gradient)
            
            if epoch % 10 == 0:
                cost = costFunction(wordFeature, targetLabel, weights)
#                 print("costFunction: ",cost)
                print(c,"                          ",cost)
        classifier[c,:] = weights

    return classifier

def calculate_accuracy(predictions,actualPOS):
    correct = 0
    for i in range(len(predictions)):
        if predictions[i] == actualPOS[i]:
            correct += 1
    
    correct /= len(predictions)
    print("Accuracy :",correct*100 , "%")

def evaluate(weights,testdata,actualPOS):
    actualPOS = np.array(actualPOS)

    scores = np.dot(testdata, weights.T)
    scores = np.round(sigmoid(scores))
    predictions = scores.argmax(axis=1)	
    calculate_accuracy(predictions,actualPOS)
    

In [71]:
########################## PREDICTING TAGS ########################
def preprocess_sent(test_sent):
    word_feature = []
    sentence = test_sent.split(" ")
    
    for token_index, token_pair in enumerate(sentence):
        word_feature.append(get_feature(token_pair,token_index,test_sent))
    
    return sentence,word_feature

def predict_POS(features,weights):
    scores = np.dot(features,weights.T)
    scores = np.round(sigmoid(scores))
    predictions = scores.argmax(axis=1)
    
    return predictions

def pred_tag_sequence(test_sent,weights):
    global tag_num_dict
    estimatedPOS = []
    num_tag_dict = {}
    
    sentence,word_feature = preprocess_sent(test_sent)
    vectorized_features = v.transform(word_feature)
    predictions = predict_POS(vectorized_features,weights)
    
    for key,val in tag_num_dict.items():
        num_tag_dict[val] = key
        
    for index,sent in enumerate(sentence):
        estimatedPOS.append((sent,num_tag_dict.get(predictions[index],len(num_tag_dict)-1)))
        
    print("Predicted POS Tags: ",estimatedPOS)
    

In [72]:
############################# MAIN #############################
for i in raw_data:
    data.append(i)
processed_data = pre_process(data)

train_data = []
test_data = []

for i in range(len(processed_data)):
    if i <= 0.8*len(processed_data):
        train_data.append(processed_data[i])
    else:
        test_data.append(processed_data[i])

build_dicts(train_data)
features,pos_labels = feature_extraction(train_data)


print(len(Vocab))
print(len(tag_list))
print(tag_list)

624
29
{'TO', 'RP', 'CS', 'NN', 'PP', 'AP', 'WR', 'AT', 'PN', 'RB', 'DO', 'QL', 'AB', 'NR', 'HV', 'OD', 'WD', 'MD', 'IN', 'CD', 'CC', 'EX', 'NP', '.', 'VB', 'DT', 'WP', 'JJ', 'BE'}


In [76]:
############################# TRAINING #############################
v = DictVectorizer(sparse=False)
train_x = v.fit_transform(features)
weights = logistic_Regression(train_x,pos_labels,5,0.1)


Training for label           Cost Function

0                            396.93594038934859178
1                            29.875715612838742474
2                            644.3700331106500838
3                            2127.2854119779570032
4                            683.3259904137802483
5                            117.03471020763982717
6                            146.4587713416905014
7                            517.4728822088071694
8                            19.531349732243472409
9                            417.9639697065304384
10                            30.360584257222641142
11                            43.56896223386971269
12                            42.108836581550017804
13                            201.6092389526472931
14                            169.31884620043930342
15                            37.560043922638565558
16                            75.91352871063359986
17                            251.31008106512473668
18                            944.2570

In [77]:
############################# TESTING #############################
features1,pos_labels1 = feature_extraction(test_data)
test_x = v.transform(features1)
evaluate(weights,test_x,pos_labels1)


Accuracy : 62.22222222222222 %


In [ ]:
############################# PREDICTING TAG SEQUENCE #############################
test_sent = input("Enter Sentence: ")
pred_tag_sequence(test_sent,weights)
